In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

class Node:
  def __init__(self):
    self.child = [None, None]
  
  def __setitem__(self, k: bool, _):
    if self.child[k] is None: self.child[k] = Node() 
  
  def __getitem__(self, k: bool):
    if self.child[k]: return self.child[k]
    self.child[k] = Node()
    return self.child[k]
  
  def __contains__(self, k: bool):
    return self.child[k] is not None

class BinaryTrie:
  def __init__(self, depth=32):
    self.root = Node()
    self.depth = depth

  def __contains__(self, n) :
    cur = self.root
    for i in reversed(range(self.depth)) :
      v = (n >> i) & 1
      if cur.child[v] is None : return False
      cur = cur[v]
    return True
  
  def __repr__(self): #print 모든 leaf노드의 값 출력. O(2^N) 이지만 보통 sparse하므로 보통 O(|leaf nodes|)
    cur = self.root
    if all(cur.child) : return "[]"
    S = [(cur, 0, 0)]
    res = []
    while S :
      u, d, v = S.pop()
      if not u.child[0] and not u.child[1] and d == self.depth:
        res.append(str(v))
        continue
        
      for i in range(2) :
        if u.child[i] :
          S.append((u.child[i], d+1, v*2+i))
      
    return f"[{' '.join(res)}]"

  def add(self, n):
    cur = self.root
    for i in reversed(range(self.depth)) :
      v = (n >> i) & 1
      cur[v] = None
      cur = cur[v]
  
  def remove(self, n) : #O(self.depth)
    cur = self.root
    path = [cur] #자식 노드가 없어진 노드들을 확인하기 위해 경로 저장
    for i in reversed(range(self.depth)) :
      v = (n >> i) & 1
      if cur.child[v] is None : return False #존재하지 않는 노드를 삭제하려고 할 때
      cur = cur[v]
      path.append(cur)

    #TODO: implement backtrack to remove unused nodes

    return True

  def query(self, n): # O(self.depth), trie 상에 존재하는 원소중에 n과 XOR 연산했을 때의 최솟값
    cur = self.root
    res = 0
    for i in reversed(range(self.depth)) :
      v = not (n >> i) & 1
      if v in cur :
        cur = cur[v]
      else :
        res += 1 << i
        cur = cur[not v]
    return res

def sol() :
  N, M = map(int, input().split())
  L = set(map(int, input().split()))
  Q = []
  for _ in range(M) :
    Q.append(int(input()))
  trie = BinaryTrie(24)

  MAX, MIN = -1, 300000 + 7777
  for v in L: 
    trie.add(v)
    MAX = max(MAX, v)
    MIN = min(MIN, v)
    
  offset = MAX - MIN + 1 #구간 [MIN, MAX] 사이에 존재하지 않는 값 중 가장 작은 값이 최솟값으로 부터 얼마나 떨어져 있는지의 offset, 즉 MIN + offset 이 그러한 최솟값이다.
  for i in range(0, MAX-MIN+1) :
    if MIN + i not in L :
      offset = i
      break

  # naive
  L2 = L.copy()
  for q in Q :
    tmp = []
    for v in L2 :
      tmp.append(q ^ v)
    L2 = tmp
    print(tmp)

  delta = []
  x = 0
  for q in Q :
    x ^= q
    res = trie.query(x)
    delta.append(res - MAX) #원본 수열로부터 얼만큼 움직였는지 저장

  ans = []
  for d in delta: 
    assert MIN + d >= 0
    if d > 0 : #문제의 정의에 의해 수열의 어떤 값도 0보다 항상 크다. 수열이 기존 수열보다 오른쪽으로 이동했다면, 수열의 최솟값이 0보다 오른쪽에 있다는 뜻이다.
      ans.append(0)
    elif MIN + d > 0 : #수열이 왼쪽으로 했거나, 그대로 있었지만 그 최솟값이 0보다 크다면 mex(L)은 0이다.
      ans.append(0)
    else: #문제에 정의에 의해 이 경우 MIN + d == 0, offset이 정답이다.
      ans.append(offset) 

  sys.stdout.write('\n'.join(map(str, ans)))
  
sol()

### 틀린 관찰
- 수열에 각 원소에 대해 xor했을 때, 순서가 바뀔지언정 전체적인 수열의 위치가 특정 위치로 옮겨진다는 관찰이 있었지만, \
수열에 0이 존재하거나, 수열 내에있는 값과 XOR하면(0이 되니까) 반례가 생겨서 실패한 풀이이다.

In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

class Node:
  def __init__(self):
    self.child = [None, None]
  
  def __setitem__(self, k: bool, _):
    if self.child[k] is None: self.child[k] = Node() 
  
  def __getitem__(self, k: bool):
    if self.child[k]: return self.child[k]
    self.child[k] = Node()
    return self.child[k]
  
  def __contains__(self, k: bool):
    return self.child[k] is not None

class BinaryTrie:
  def __init__(self, depth=32):
    self.root = Node()
    self.depth = depth

  def __contains__(self, n) :
    cur = self.root
    for i in reversed(range(self.depth)) :
      v = (n >> i) & 1
      if cur.child[v] is None : return False
      cur = cur[v]
    return True
  
  def __repr__(self): #print 모든 leaf노드의 값 출력. O(2^N) 이지만 보통 sparse하므로 보통 O(|leaf nodes|)
    cur = self.root
    if all(cur.child) : return "[]"
    S = [(cur, 0, 0)]
    res = []
    while S :
      u, d, v = S.pop()
      if not u.child[0] and not u.child[1] and d == self.depth:
        res.append(str(v))
        continue
        
      for i in range(2) :
        if u.child[i] :
          S.append((u.child[i], d+1, v*2+i))
      
    return f"[{' '.join(res)}]"

  def add(self, n):
    cur = self.root
    for i in reversed(range(self.depth)) :
      v = (n >> i) & 1
      cur[v] = None
      cur = cur[v]
  
  def remove(self, n) : #O(self.depth)
    cur = self.root
    path = [cur] #자식 노드가 없어진 노드들을 확인하기 위해 경로 저장
    for i in reversed(range(self.depth)) :
      v = (n >> i) & 1
      if cur.child[v] is None : return False #존재하지 않는 노드를 삭제하려고 할 때
      cur = cur[v]
      path.append(cur)

    #TODO: implement backtrack to remove unused nodes

    return True

  def query(self, n): # O(self.depth), trie 상에 존재하는 원소중에 n과 XOR 연산했을 때의 최솟값
    cur = self.root
    res = 0
    for i in reversed(range(self.depth)) :
      v = (n >> i) & 1
      if v in cur : #prefer same bit
        cur = cur[v]
      else :
        res += 1 << i
        cur = cur[not v]
    return res

MAX = 300000
def sol() :
  N, M = map(int, input().split())
  L = set(map(int, input().split()))
  Q = []
  for _ in range(M) :
    Q.append(int(input()))
  trie = BinaryTrie(20)
  trie2 = BinaryTrie(20) #빈칸에 대한 trie

  for i in range(MAX + 1): 
    if i in L :
      trie.add(i)
    else :
      trie2.add(i)

  ans = []
  x = 0
  for q in Q :
    x ^= q
    match x :
      case _ if trie.query(x) : # 수열의 최솟값이 0보다 크므로 mex(L)은 0이다.
        ans.append(0)
      case _:
        # debug(st.L[1])
        ans.append(trie2.query(x))

  sys.stdout.write('\n'.join(map(str, ans)))
  
sol()

### 틀린 풀이
- xor했을 때 최솟값을 출력하(는 것이라고 굳게 믿고 이틀동안 왜 WA가 계속 나올지 고민하게 하)는 쿼리를 구현한 뒤, \
이진 트라이 2개를 만들고나서 한 개는 $L$ 에 존재하는 원소를 집어넣고, 반대쪽엔 존재하지 않는 원소를 집어넣는다.\
각 트라이를 A, B라고 했을 때, A의 쿼리 값이 0이 아니라면, 0을 넣었을 때 $mex(L)$ 이 성립하므로 0이 정답이다.\
0이라면 B의 query를 수행한 값을 출력한다.
- B에 들어간 원소들도 XOR해서 이동할 것이라는 믿음으로 제출한 마지막 코드였지만, 3%에서 가차없이 WA가 났다.
- query의 작동이 xor 했을 때 최솟값을 반환하는 쿼리가 맞는지 확실하지 않았고, 위의 믿음도 틀렸을 수 있다.

;; 0 다음에 연속하는 숫자가 없다면
2 1
0 2
1
;; 정답: 1
4 1
1 2 3 4
1
;; 정답: 3
4 1
0 1 3 4
1
;; 2 XOR를 해서 mex가 바뀌는 경우
3 1
0 2 3
3
;; 정답:0, 쿼리에 0이 포함되는 경우 w/o 0 in L
5 1
1 2 3 4 5
0
;; 정답:5, 쿼리에 0이 포함되는 경우 w/ 0 in L
5 1
0 1 2 3 4
0

### 풀이(Codeforce round 430 div.2 - D. Vitya and Strange Lesson)
- 문제의 쿼리 $Q(x)$ 는 $L' = \{v \oplus x \mid v \in L\}$ 인 $L'$ 에 대해 $\text{mex}(L')$ 를 반환하는 것이다.
  - `mex`(minimum excludant)는 Array $L$ 에 대해 다음과 같이 정의된다. $\text{mex}(L) = \min\{m \in \mathbb{N} \mid m \notin L\}$
- (자력 관찰) 마지막에 적용한 쿼리를 $x_i$ 라고 하고, 그 다음에 오는 쿼리를 $x_{i+1}$ 이라고 하자\
$Q$ 에 대해 $L$ 을 직접 수정하지 않고 $Q(x_i \oplus x_{i+1})$ 을 대신 쿼리하면 동일한 결과를 얻을 수 있다.\
따라서 매번 쿼리마다 xor 연산을 누적해주는 것으로 배열 수정 오버헤드를 없앨 수 있다.
- 이진 트라이의 쿼리 $Q(x)$ 를 다음과 같이 구현할 수 있다.
  - `res = 0` 으로 초기화 한뒤 $x$ 의 각 비트를 거꾸로 순회하면서 excludant를 탐색한다.
    - 현재 비트 $v$ 로 향하는 자식 노드가 존재하지 않으면 탐색을 종료한다.
      - $v$ 를 선택하고 나머지의 하위 노드들을 모두 0으로 골라도 자리가 항상 남아있기 때문이다.
    - $v$ 로 향하는 자식이 완전 이진 트리라면 $v := \lnot v$ 로 설정하고, res를 $i << k$ 만큼 증가시킨다. $k$ 는 현재 비트의 위치
    - $v$ 를 선택하고 $v$ 로 향하는 자식 노드로 이동한다.
  - `res`를 반환한다.
- 위 쿼리 구현을 위해, trie에 노드를 추가할 때, 서브 트리의 개수를 1개씩 늘리도록 구현할 수 있다.
  - 세그먼트 트리마냥 $O(N \log N)$ 으로 할 필요 없이, $O(N)$ 으로 구현할 수 있다. 구현 참고.
- 전반적으로 이진 트라이의 `xor했을 때의 최댓값 쿼리`에 집착하느라 시간을 많이 날렸다.

### 구현
- 시간복잡도: $O(\log 300000 \cdot N + M \log 300000) = O(20(N + M))$
  - $\log 300000$ 은 $A_i$ 의 범위의 최댓값으로, 20개 미만의 비트로 나타낼 수 있어서 나온 값이다.
- 이진 트라이의 삭제 연산만 좀 구현하면 템플릿화 할 수 있을 것 같은데 잘 안된다..

In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

class Node:
  def __init__(self):
    self.child = [None, None]
    self.n = 0 #number of subtree nodes
  
  def add(self, k: bool) :
    if self.child[k] is None: self.child[k] = Node() 

  def __getitem__(self, k: bool):
    if self.child[k]: return self.child[k]
    self.child[k] = Node()
    return self.child[k]
  
  def __contains__(self, k: bool):
    return self.child[k] is not None

class BinaryTrie:
  def __init__(self, depth=32):
    self.root = Node()
    self.depth = depth

  def __contains__(self, n) :
    cur = self.root
    for i in reversed(range(self.depth)) :
      v = (n >> i) & 1
      if cur.child[v] is None : return False
      cur = cur[v]
    return True
  
  def print_all(self): #print 모든 leaf노드의 값 출력. O(2^N) 이지만 보통 sparse하므로 보통 O(|leaf nodes|)
    cur = self.root
    if all(cur.child) : return "[]"
    S = [(cur, 0, 0)]
    res = []
    while S :
      u, d, v = S.pop()
      if not u.child[0] and not u.child[1] and d == self.depth:
        res.append(str(v))
        continue
        
      for i in range(2) :
        if u.child[i] :
          S.append((u.child[i], d+1, v*2+i))
      
    return f"[{' '.join(res)}]"

  def insert(self, n): #assume no duplicates, O(self.depth)
    cur: Node = self.root
    for i in reversed(range(self.depth)) :
      v = (n >> i) & 1
      cur.add(v)
      cur.n += 1
      cur = cur[v]
    cur.n += 1

  def remove(self, n) : #O(self.depth)
    cur = self.root
    path = [cur] #자식 노드가 없어진 노드들을 확인하기 위해 경로 저장
    for i in reversed(range(self.depth)) :
      v = (n >> i) & 1
      if cur.child[v] is None : return False #존재하지 않는 노드를 삭제하려고 할 때
      cur = cur[v]
      path.append(cur)

    #TODO: implement backtrack to remove unused nodes

    return True

  def query(self, n): # O(self.depth), xor mex
    cur = self.root
    res = 0
    for i in reversed(range(self.depth)) :
      v = (n >> i) & 1
      if not cur.child[v] : #if no child is found upon selecting this, there is free place to fill by selecting 0
        break
      elif cur.child[v].n == (1 << i) : #Check if subtree is fully populated
        cur = cur[not v] 
        res += 1 << i
      else : #Then, there is possible a place to be filled
        cur = cur[v]
    return res

MAX = 300000
def sol() :
  N, M = map(int, input().split())
  L = set(map(int, input().split()))
  Q = []
  for _ in range(M) :
    Q.append(int(input()))
  trie = BinaryTrie(20)

  for v in L: 
    trie.insert(v)

  ans = []
  x = 0
  for q in Q :
    x ^= q
    ans.append(trie.query(x))

  sys.stdout.write('\n'.join(map(str, ans)))
  
sol()